In [1]:
""" OBJECT TRACKING """
# is the process of locating a moving object
# Types of OT 
""" OPTICAL FLOW """
# 1. Will take given set of points and frames
# 2. It will attempt to find those points in next frame
# 3. It's up to user to supply the point to track by using object detection  

' OPTICAL FLOW '

In [2]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt

In [3]:
corner_track_params = dict(maxCorners = 10, qualityLevel = 0.3, minDistance = 7, blockSize = 7)

In [4]:
#### Parameters for Lucas Kanade Optical Flow ####
lk_params = dict(winSize = (200,200), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10,0.03))
# level -> represent different reslution

In [6]:
# CAPUTER VIDEO 
cap = cv2.VideoCapture(0)

# GRAB THE VERY FIRST FRAME OF THE CAMERA
ret ,previous_frame = cap.read()

# APPLYING GRAY SCALE
previous_gray = cv2.cvtColor(previous_frame, cv2.COLOR_BGR2GRAY)

# SELECT POINTS WANT TO BE TRACK
previous_points = cv2.goodFeaturesToTrack(previous_gray, mask = None, **corner_track_params)

# CREATE A MATCHING MASK OF THE PREVIOUS FRAME FOR DRAWING ON LATER
mask = np.zeros_like(previous_frame)

while True:
    
    # GRAB THE CURRENT FRAME
    ret, frame = cap.read()
    
    # APPLYING GRAY SCALE
    frame_gray = cv2.cvtColor(frame ,cv2.COLOR_BGR2GRAY)
    
    # CALCULATE OPTICAL FLOW ON GRAY SCALE IMAGE
    # RETURN NEXT POINT IN CURRENT FRAME
    next_points, status, err = cv2.calcOpticalFlowPyrLK(previous_gray,frame_gray, previous_points,None, **lk_params)
    
    # Using the returned status array (the status output)
    # status output status vector (of unsigned chars); each element of the vector is set to 1 if
    # the flow for the corresponding features has been found, otherwise, it is set to 0.
    good_new = next_points[status ==1]
    good_prev = previous_points[status ==1]
    
    # USE RAVEL TO GET POINTS TO DRAW LINES AND CIRCLES
    for i, (new,prev) in enumerate(zip(good_new, good_prev)):
        
        x_new ,y_new = new.ravel()
        x_prev ,y_prev = prev.ravel()
        
        # create line from previous farme to cuurent frame
        mask = cv2.line(mask ,(x_new ,y_new) ,(x_prev,y_prev) ,(0,255,0) ,3)
        
        # create circle on current frame
        frame = cv2.circle(frame,(x_new ,y_new) ,8 ,(255,0,0) ,-1)
        
    img = cv2.add(frame, mask)
    cv2.imshow('tracking', img)
    
    
    # CLOSE WINDOW
    k = cv2.waitKey(30) & 0xFF
    
    if k == 27:
        break
     
    # UPDATE PREVIOUS FRAME AND PREVIOUS POINT
    previous_gray = frame_gray.copy()
    previous_points = good_new.reshape(-1,1,2)
    
    
cv2.destroyAllWindows()    
cap.release()

In [7]:
# we note that it's very good in slow tracking but with fast tracking may lose some of points

In [8]:
""" DENSE OPTICAL FLOW """

' DENSE OPTICAL FLOW '

In [9]:
# CAPUTER VIDEO 
cap = cv2.VideoCapture(0)

# GRAB THE VERY FIRST FRAME OF THE CAMERA
ret ,frame1 = cap.read()

# APPLYING GRAY SCALE 
prvsImg = cv2.cvtColor(previous_frame, cv2.COLOR_BGR2GRAY)

# MAKE MASK IN HSV COLOR
hsv_mask = np.zeros_like(frame1)
hsv_mask[:,:,1] = 255

while True:
    
    # GRAB THE CURRENT FRAME
    ret, frame2 = cap.read()
    
    # APPLYING GRAY SCALE
    nextImg = cv2.cvtColor(frame2 ,cv2.COLOR_BGR2GRAY)
    
    # CHECK OUT THE MARKDOWN TEXT ABOVE FOR A BREAK DOWN OF THESE PARAMETER, MOST OF THESE ARE JUST SUGESSTED  DEFAULTS
    flow = cv2.calcOpticalFlowFarneback(prvsImg,nextImg, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # CONVERT FLOW TO POLAR CORDINATE (MAGINTUDE AND ANGLE)
    mag, ang = cv2.cartToPolar(flow[:,:,0], flow[:,:,1],angleInDegrees=True)
    hsv_mask[:,:,0] = ang/2
    hsv_mask[:,:,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    
    # CONVERT BACK TO BGR TO SHOW WITH IMSHOW FROM CV
    bgr = cv2.cvtColor(hsv_mask,cv2.COLOR_HSV2BGR)
    cv2.imshow('frame2',bgr)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
        
     # SET THE PREVIOUS IMAGE AS THE NEXT IMAGE FOR THE LOOP
    prvsImg = nextImg

    
cap.release()
cv2.destroyAllWindows()    